In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/sample_submission.csv
/kaggle/input/dataset/train.csv
/kaggle/input/dataset/test.csv


# Données

In [2]:
import tensorflow as tf

In [3]:
train = pd.read_csv('../input/dataset/train.csv')
test = pd.read_csv('../input/dataset/test.csv')
sample = pd.read_csv('../input/dataset/sample_submission.csv')

### Données d'entraînement

In [4]:
train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [5]:
from datasets import load_dataset

In [6]:
#dataset = load_dataset('MayaGalvez/multilingual_xnli')
dataset = load_dataset('multi_nli')

  0%|          | 0/3 [00:00<?, ?it/s]

J'ai décidé d'utiliser un dataset multi_nli parce que c'est en anglais et qu'il y a beaucoup de données correspondantes à ce que l'on souhaite avoir pour l'entraînement

In [7]:
nliBefore = []
for t in dataset['train']:
    t1, t2, t3 = t['premise'], t['hypothesis'], t['label']
    if t1 and t2 and t3 in {0,1,2}:
        nliBefore.append((t1,t2,t3,'en'))
nli = pd.DataFrame(nliBefore, columns=['premise', 'hypothesis', 'label', 'lang'])

J'ai ajouté lang si jamais on vient à adapter à d'autres langues également

In [8]:
nli.head()

,premise,hypothesis,label,lang
0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,1,en
1,you know during the season and i guess at at y...,You lose the things to the following level if ...,0,en
2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,0,en
3,How do you know? All this is their information...,This information belongs to them.,0,en
4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,1,en


Au dessus, voici la signification des labels selon le nli utilisé :
- label 0 : redondance
- label 1 : neutre
- label 2 : contradiction

In [9]:
nli.label.value_counts()

2    130903
1    130900
0    130899
Name: label, dtype: int64

In [10]:
nli =  nli.groupby('label').apply(lambda x: x.sample(n=5000)).reset_index(drop = True)

A priori pas vraiment d'oversampling ni d'undersampling nécessaire    
Mais ici, je n'ai quand même gardé que 5000 entrées pour les données de chaque label parce que j'ai des problèmes de mémoire et j'essaie de trouver une solution

In [11]:
train = pd.concat([nli[['premise','hypothesis','label']], train[['premise','hypothesis','label']][train['lang_abv'] == 'en']], axis = 0)

Pas sûr d'être obligé de faire ce qu'il y a au dessus, c'est surtout pour utiliser également les données qui étaient sur le kaggle d'origine, on en avait déjà beaucoup donc ça sert probablement pas à grand chose.    
    
Sinon juste faire un train = nli

In [12]:
train.label.value_counts()

0    7427
2    7277
1    7166
Name: label, dtype: int64

In [13]:
train.isnull().sum()

premise       0
hypothesis    0
label         0
dtype: int64

Aucune valeur nulle, ce qui nous arrange

In [14]:
train

,premise,hypothesis,label
0,um i haven't kept up with uh the Texas politic...,There was a governor fight in Texan politics,0
1,Is it possible that she could have swallowed t...,Is it possible that she could have taken the p...,0
2,"The views climbing up the steep, narrow road a...",There are stunning views to be seen after clim...,0
3,at the firm,In the office.,0
4,cause i'm in Garland,Since I am located in Garland.,0
...,...,...,...
12115,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,2
12116,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,0
12117,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",2
12118,At the west end is a detailed model of the who...,The model temple complex is at the east end.,2


In [15]:
train['all'] = train['premise'] + ' [SEP] ' + train['hypothesis']

[SEP] aiderait le modèle à comprendre quel token fait partie de quel phrase entre les deux d'origine

In [16]:
train = train.drop(['premise','hypothesis'], axis=1)

In [17]:
train

,label,all
0,0,um i haven't kept up with uh the Texas politic...
1,0,Is it possible that she could have swallowed t...
2,0,"The views climbing up the steep, narrow road a..."
3,0,at the firm [SEP] In the office.
4,0,cause i'm in Garland [SEP] Since I am located ...
...,...,...
12115,2,The results of even the most well designed epi...
12116,0,But there are two kinds of the pleasure of do...
12117,2,The important thing is to realize that it's wa...
12118,2,At the west end is a detailed model of the who...


Voilà a priori à quoi doivent ressembler les données d'entraînement pour le modèle

### Données de test

In [18]:
test = test[['premise','hypothesis']][test.lang_abv == 'en']
test['all'] = test['premise'] + ' [SEP] ' + test['hypothesis']
test = test.drop(['premise','hypothesis'], axis=1)

# Modèle

In [19]:
from transformers import BertTokenizer, TFXLMRobertaModel, AutoTokenizer,TFAutoModel

2022-10-11 17:17:26.364527: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [20]:
model_roBerta ='joeddav/xlm-roberta-large-xnli'
tokenizer = AutoTokenizer.from_pretrained(model_roBerta)

In [21]:
maxLen = max(len(x.split()) for x in train['all'])

bert_encoder = TFXLMRobertaModel.from_pretrained(model_roBerta)
input_word_ids = tf.keras.Input(shape=(maxLen,), dtype=tf.int32, name="input_word_ids")
embedding = bert_encoder(input_word_ids)[0]
x = tf.keras.layers.GlobalAveragePooling1D()(embedding)
x = tf.keras.layers.LayerNormalization()(x)
output = tf.keras.layers.Dense(3, activation="softmax")(x)

model = tf.keras.models.Model(inputs=input_word_ids, outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

Some layers from the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing TFXLMRobertaModel: ['classifier']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at joeddav/xlm-roberta-large-xnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


In [22]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 283)]             0         
_________________________________________________________________
tfxlm_roberta_model (TFXLMRo TFBaseModelOutputWithPool 559890432 
_________________________________________________________________
global_average_pooling1d (Gl (None, 1024)              0         
_________________________________________________________________
layer_normalization (LayerNo (None, 1024)              2048      
_________________________________________________________________
dense (Dense)                (None, 3)                 3075      
Total params: 559,895,555
Trainable params: 559,895,555
Non-trainable params: 0
_________________________________________________________________


# Entraînement

In [23]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categories='auto')
array_hot_encoded = ohe.fit_transform(train[['label']]).toarray()
data_hot_encoded = pd.DataFrame(array_hot_encoded, index=train.index)
data_other_cols = train.drop(columns='label')
train = pd.concat([data_hot_encoded, data_other_cols], axis=1)
train.columns = train.columns.astype(str)
train['1'] = train['1'].astype("category").cat.codes
train['0'] = train['0'].astype("category").cat.codes
train['2'] = train['2'].astype("category").cat.codes

Aucune idée de s'il faut vraiment faire ce qu'il y a juste au dessus, je l'ai mis pour tester des trucs, j'ai que du "Your notebook tried to allocate more memory than is available. It has restarted", je ne vois perso pas l'intérêt de faire un one-hot ici, à voir

In [29]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    
batchSize = 16*strategy.num_replicas_in_sync

In [30]:
train_input = tokenizer.batch_encode_plus(train['all'].values.tolist(), padding='max_length', max_length=maxLen, truncation=True)['input_ids']
train_final = (tf.data.Dataset.from_tensor_slices((train_input,train[['0','1','2']])).repeat().batch(batchSize).prefetch(tf.data.experimental.AUTOTUNE))

In [31]:
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping

In [ ]:
history = model.fit(train_final, steps_per_epoch= train.shape[0]//batchSize, batch_size = batchSize, epochs=3, verbose = 1, shuffle= True)

Epoch 1/3


2022-10-11 17:31:51.012193: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


# Prédiction

In [ ]:
testing_input = tokenizer.batch_encode_plus(test['all'].values.tolist(), padding='max_length', max_length=maxLen, truncation=True)['input_ids']
test_final = (tf.data.Dataset.from_tensor_slices(testing_input).batch(batchSize))

In [ ]:
pred = model.predict(test_final)

In [ ]:
pred_df = pd.DataFrame()
pred_df['prediction'] = [x.argmax() for x in pred]

In [ ]:
pred_df